# Alaska Satellite Facility

Use the ASF Search API by extent and year.

Documentation is available here: https://docs.asf.alaska.edu/api/keywords/


In [1]:
import requests
import geopandas as gpd
import shapely

from utils import get_australia_coords

url = "https://api.daac.asf.alaska.edu/services/search/param?"

In [5]:
def search(start, end):
    coords = get_australia_coords()
    coords_string = ",".join([f"{x[0]} {x[1]}" for x in coords])
    polygon = f"polygon(({coords_string}))"

    params = {
        "platform": "S1",
        "processingLevel": "GRD_HD",
        "output": "JSON",
        "start": start,
        "end": end,
        "maxResults": 12000,
        "flightDirection": "DESCENDING",
        "beamMode": "IW",
        "intersectsWith": polygon
    }
    response = requests.get(url, params=params)
    return response.json()[0]

In [6]:
year_count = {}
for year in range(2014, 2025):
    start = f"{year}-01-01T00:00:00Z"
    end = f"{year}-12-31T23:59:59Z"
    features = search(start, end)
    year_count[year] = len(features)
    print(year, len(features))

2014 54
2015 1592
2016 3017
2017 8911
2018 9069
2019 9214
2020 8958
2021 8676
2022 5560
2023 5612
2024 1096


In [ ]:
geojson = []
for feature in features:
    feature["geometry"] = shapely.from_wkt(feature["stringFootprint"])

gdf = gpd.GeoDataFrame(features).set_geometry('geometry', crs='EPSG:4326')
gdf.explore(style_kwds=dict(fillOpacity=0.05))
